# Minimizing Tracking Error (Custom Function in pyPortfolioAnalysis)

In [1]:
import pandas_datareader as pdr

In [2]:
assets = ['DEVIX',
'FSDPX',
'MEIIX',
'ODVIX',
'OPPAX',
'PDBAX',
'PRGTX',
'PRNHX',
'AEPGX',
'AGTHX',
'PRDGX',
'FBNRX',
'VEVRX',
'VIEIX',
'VINIX',
'VFAIX',
'VGHCX',
'VFTNX',
'VGSNX',
'RFDTX',
'RFFTX',
'RFHTX',
'RFKTX']
bench_name = ['FXAIX']

In [3]:
port = [0]*len(assets)
for i in range(0, len(assets)):
    port[i] = pdr.get_data_yahoo(assets[i]).iloc[:,5].pct_change().dropna()

In [4]:
import pandas as pd
port_ret = pd.concat(port, axis = 1)
port_ret.columns = assets
port_ret.head()

,DEVIX,FSDPX,MEIIX,ODVIX,OPPAX,PDBAX,PRGTX,PRNHX,AEPGX,AGTHX,...,VIEIX,VINIX,VFAIX,VGHCX,VFTNX,VGSNX,RFDTX,RFFTX,RFHTX,RFKTX
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-22,0.021381,0.022500,0.016871,0.027872,0.026900,0.000000,0.026029,0.021682,0.029998,0.020452,...,0.024884,0.020315,0.020141,0.021063,0.020851,0.027590,0.013736,0.018800,0.019264,0.019481
2016-01-25,-0.025075,-0.044173,-0.013663,-0.006596,-0.015600,0.000711,-0.013093,-0.016563,-0.009549,-0.015032,...,-0.021135,-0.015665,-0.022498,-0.009704,-0.015523,-0.006563,-0.008130,-0.010545,-0.011168,-0.011323
2016-01-26,0.027135,0.012790,0.017480,0.007008,0.012558,0.000710,0.006633,0.009474,0.011569,0.010977,...,0.020342,0.014165,0.019728,0.004777,0.013278,0.021622,0.009107,0.010657,0.011295,0.011453
2016-01-27,-0.005743,-0.011955,-0.002593,-0.003663,-0.009892,0.000710,-0.022241,-0.013816,-0.004765,-0.012447,...,-0.013466,-0.010806,-0.005067,-0.014458,-0.011466,-0.019401,-0.003610,-0.006151,-0.006873,-0.007077
2016-01-28,0.003697,0.001193,0.001950,0.006985,-0.004772,0.000000,0.005055,-0.007930,-0.003112,0.006436,...,-0.002482,0.005578,0.001389,-0.022378,0.001657,-0.007194,0.003623,0.003537,0.003460,0.003564


In [5]:
bench = pdr.get_data_yahoo(bench_name).iloc[:,4].pct_change().dropna()

In [6]:
import numpy as np
def tracking_error(w, R, Rb):
    ret_p = list(np.matrix(w).dot(np.matrix(R).T))
    alpha = list(Rb) - ret_p[0]
    te = np.sqrt(np.var(alpha))
    return(te)

In [32]:
from pyPortfolioAnalysis import *
port1 = portfolio_spec(assets = assets)
add_constraint(port1, kind = 'long_only')
add_constraint(port1, kind = 'full_investment')
add_objective(port1, kind = 'return',name = 'mean' ,target = 0.0005)
add_objective(port1, kind = 'performance_metrics', name = {'te':tracking_error}, arguments = {'Rb':bench},
             target = 0.0)

In [36]:
optimize_portfolio(port_ret, port1, optimize_method = 'dual_annealing', itersize = 250)

[{'weights': {'DEVIX': 0.0014899679597321523,
   'FSDPX': 0.08156312349002337,
   'MEIIX': 0.013694131659061779,
   'ODVIX': 0.04133695793923222,
   'OPPAX': 0.059064554512264784,
   'PDBAX': 0.051906213551857545,
   'PRGTX': 0.005046677420578913,
   'PRNHX': 0.0450530373803542,
   'AEPGX': 0.10199258142208832,
   'AGTHX': 0.06416221186704164,
   'PRDGX': 0.01764525337163289,
   'FBNRX': 0.0009565196655483635,
   'VEVRX': 0.006551510625773701,
   'VIEIX': 0.022857752430427366,
   'VINIX': 0.09390623358835798,
   'VFAIX': 0.051100238328960344,
   'VGHCX': 0.05906960129544654,
   'VFTNX': 0.03138439579355551,
   'VGSNX': 0.039467071270922104,
   'RFDTX': 0.10101353478215498,
   'RFFTX': 0.0120400478343418,
   'RFHTX': 0.021397660225566755,
   'RFKTX': 0.07730071957910101}},
 {'objective_measures': [['return', array(0.00051895)],
   ['performance_metrics', array(0.)]]},
 {'best': 1.81004174265616}]

In [37]:
opt_weights = extract_weights(port1)
opt_port_ret = np.matrix(opt_weights).dot(port_ret.T).tolist()

In [38]:
alpha = bench-opt_port_ret[0]

In [39]:
np.sqrt(np.var(alpha))

0.003039780407780884